In [1]:
import plotly
import pandas as pd
import plotly as py
import csv
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import plotly.figure_factory as ff
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
from plotly import __version__
import plotly.offline as pyo
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from statistics import mean
import chart_studio.plotly as py

#py.offline.init_notebook_mode(connected=True)

In [2]:
import chart_studio
chart_studio.tools.set_credentials_file(username='mathichan', api_key='ORDAD9bTDuqUnr7x2Qvm')

In [3]:
import csv

measurements = []
data = {}
day_values = 1
keys = []
days = {}
with open('bold_data.csv', 'r') as readFile:
    reader = csv.reader(readFile, delimiter=";")
    i = 0
    for row in reader:
        if i >= 1:
            g_force = row[2].replace(',', '.')
            g_force = round(float(g_force), 2)
            row[2] = g_force
            voltage = row[3].replace(',', '.')
            voltage = round(float(voltage), 2)
            row[3] = voltage
            temp = row[4].replace(',', '.')
            temp = round(float(temp), 2)
            row[4] = temp

            # only add if enough g_force
            if 1.1 < g_force < 3:
                measurements.append(row)
        if i == 0:
            data = {i: [] for i in row}
            keys = row
            i = 2

for measurement in measurements:
    if day_values == 1:
        day_values = {i: [] for i in keys}

    i = 0
    for key in keys:
        data.get(key).append(measurement[i])
        i += 1

    i = 0
    for key in keys:
        if days.get(measurement[0]) is not None:
            day_values.get(key).append(measurement[i])
            days[measurement[0]] = day_values
        else:
            day_values = {i: [] for i in keys}
            day_values.get(key).append(measurement[i])
            days.update({measurement[0]: day_values})
        i += 1

df = pd.DataFrame(data)
display(df)

Date      Time  G-Force  Voltage  Temperature
0     15-05-2020  09:01:59     1.11     4.19        30.91
1     15-05-2020  09:02:00     1.21     4.19        30.90
2     15-05-2020  09:02:07     1.15     4.19        30.82
3     15-05-2020  09:02:20     1.31     4.19        30.64
4     15-05-2020  09:02:22     1.14     4.19        30.63
...          ...       ...      ...      ...          ...
4121  03-06-2020  08:00:54     1.24     3.32        27.76
4122  03-06-2020  08:01:07     1.34     3.32        27.75
4123  03-06-2020  08:01:08     1.12     3.33        27.76
4124  03-06-2020  08:16:39     1.52     3.32        27.39
4125  03-06-2020  08:16:43     1.12     3.32        27.40

[4126 rows x 5 columns]

# NOTE TO SELF: day/df for avg/count is here look no further

In [4]:
day = days['16-05-2020']
g_force = day['G-Force']
time = day['Time']
date = day['Date']

In [13]:
g_force = df['G-Force']
time = df['Time']
date = df['Date']

In [14]:
data_dict = {}
time = [int(i.split(':')[0]) for i in time]
data_dict = {i: {"count": 0, "avg": []} for i in range(0,24)}

for i in range(0, len(time)):
    if time[i] in data_dict.keys():
        data_dict[time[i]]["avg"].append(g_force[i])
        data_dict[time[i]]["count"] += 1

for key in data_dict.keys():
    if len(data_dict[key]["avg"]) > 0:
        data_dict[key]["avg"] = round(mean(data_dict[key]["avg"]), 2)
    else:
        data_dict[key]["avg"] = 0
        
temp = []

for key in data_dict.keys():
    temp.append({"time": key, "avg": data_dict[key]["avg"], "count": data_dict[key]["count"]})
        
hourly_df = pd.DataFrame(temp)
hourly_df

4125


time   avg  count
0      0  1.20    120
1      1  1.22      5
2      2  1.18     56
3      3  1.20    131
4      4  1.13      5
5      5  1.24     54
6      6  1.21    186
7      7  1.21     43
8      8  1.26    374
9      9  1.26    464
10    10  1.25    343
11    11  1.21    203
12    12  1.27    513
13    13  1.26    251
14    14  1.19    136
15    15  1.28    384
16    16  1.16     44
17    17  1.22    135
18    18  1.24    229
19    19  1.19     30
20    20  1.20    129
21    21  1.22    187
22    22  1.20     26
23    23  1.20     78

### Lmao virker kun på specifikke dage oh well

### Normal linje

In [8]:
x = pd.DataFrame(days['16-05-2020'])['Time']
y = df['G-Force']

layout = go.Layout(
    title='G-Force on given date'
)

fig = go.Figure(layout=layout)

fig.update_xaxes(
    rangeslider_visible=True
)

fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name='G-Force'))

### Bar on given date

In [9]:
x = pd.DataFrame(days['16-05-2020'])['Time']
y = df['G-Force']

layout = go.Layout(
    title='G-Force on given date'
)

fig = go.Figure(layout=layout)

fig.update_xaxes(
    rangeslider_visible=True
)

fig.add_trace(
    go.Bar(x=x, y=y, name="-")
)

### Graph days compared (could also be bar I guess whatevs)

In [12]:
x0 = pd.DataFrame(days['16-05-2020'])['Time']
y0 = pd.DataFrame(days['16-05-2020'])['G-Force']
y1 = pd.DataFrame(days['17-05-2020'])['G-Force']
y2 = pd.DataFrame(days['18-05-2020'])['G-Force']

layout = go.Layout(
    title='hhv. 16., 17., 18.'
)

fig = go.Figure(layout=layout)

fig.update_xaxes(
    rangeslider_visible=True
)

# Add traces
fig.add_trace(go.Scatter(x=x0, y=y0,
                    mode='lines',
                    name='16/05-20'))
fig.add_trace(go.Scatter(x=x0, y=y1,
                    #mode='lines+markers',
                    mode='lines',
                    name='17/05-20'))
fig.add_trace(go.Scatter(x=x0, y=y2,
                    #mode='lines',
                    mode='lines',
                    name='18/05-20'))

### Bar on given date w/ color

In [13]:
fig = px.bar(pd.DataFrame(days['16-05-2020']), x='Time', y='G-Force',
             hover_data=['Time', 'G-Force'], height=400, color='G-Force', text='G-Force')
fig.update_traces(texttemplate = '%{text: .2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig

### Bar on given date pr hour w/ count and shit

In [642]:
# hours = []
# count = []
# average = []

# for i in range(0, len(hourly_df['time'])):
#     if hourly_df['count'][i] != 0.0:
#         count.append(hourly_df['count'][i])
#         hours.append(hourly_df['time'][i])
#         average.append(hourly_df['avg'][i])

In [641]:
hours = []
count = []
average = []

for i in range(0, len(hourly_df['time'])):
    if hourly_df['count'][i] != 0.0:
        count.append(hourly_df['count'][i])
        hours.append(hourly_df['time'][i])
        average.append(hourly_df['avg'][i])

fig = px.bar(x=hours, y=count, hover_data=[count, average], height=420, color=count, text=average)
fig.update_traces(texttemplate = '%{text: .2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_yaxes(range=[0, 20], title_text='Count')
fig.update_xaxes(title_text='Time')

fig

### Subplots

In [600]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2)

fig.add_trace(
    go.Scatter(x=pd.DataFrame(days['16-05-2020'])['Time'], y=df['G-Force'], name="16-05-2020"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=pd.DataFrame(days['17-05-2020'])['Time'], y=df['G-Force'], name="17-05-2020"),
    row=1, col=2
)


fig.add_trace(
    go.Scatter(x=pd.DataFrame(days['18-05-2020'])['Time'], y=df['G-Force'], name="18-05-2020"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=pd.DataFrame(days['19-05-2020'])['Time'], y=df['G-Force'], name="19-05-2020"),
    row=2, col=2
)

fig.update_layout(height=600, width=800, title_text="Subplots")
fig.show()

In [599]:
from plotly.subplots import make_subplots
# throw in range on y axis

fig = make_subplots(rows=2, cols=2)

fig.add_trace(
    go.Bar(x=pd.DataFrame(days['16-05-2020'])['Time'], y=df['G-Force'], name="16-05-2020", ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=pd.DataFrame(days['17-05-2020'])['Time'], y=df['G-Force'], name="17-05-2020"),
    row=1, col=2
)


fig.add_trace(
    go.Bar(x=pd.DataFrame(days['18-05-2020'])['Time'], y=df['G-Force'], name="18-05-2020"),
    row=2, col=1
)

fig.add_trace(
    go.Bar(x=pd.DataFrame(days['19-05-2020'])['Time'], y=df['G-Force'], name="19-05-2020"),
    row=2, col=2
)

fig.update_yaxes(range=[0, 2.5])

fig.update_layout(height=600, width=800, title_text="G-Force subplots")
fig.show()

### Normal graph on given date

In [16]:
x = pd.DataFrame(days['16-05-2020'])['Time']
y = pd.DataFrame(days['16-05-2020'])['G-Force']

layout = go.Layout(
    title = 'Test'
)

fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='G-Force'))

In [10]:
layout = go.Layout(
    title='g-force over time total / size=count',
    xaxis_title="Time",
    yaxis_title="one more test again",
    plot_bgcolor='rgba(230, 230, 230, 0.2)',
    showlegend=True
)

size=hourly_df['count']

fig = go.Figure(data=[go.Scatter(
    x=hourly_df['time'],
    y=hourly_df["avg"],
    mode='markers',
    marker=dict(
        size=size,
        sizemode='area',
        sizeref=2.*max(size)/(40.**2.1),
        sizemin=4,
        color=hourly_df['avg']
    ), 
    name='Average G-Force')],
    layout=layout
)

py.iplot(fig, filename='test')

### Tbh don't even know what this is haha hwat

In [590]:
layout = go.Layout(
    title='TBD',
    plot_bgcolor='rgba(230, 230, 230, 0.2)',
    showlegend=True
)
size=hourly_df['count']

fig = go.Figure(data=[go.Scatter(
    x=hourly_df['time'],
    y=hourly_df["avg"],
    mode='markers',
    marker=dict(
        size=size,
        sizemode='area',
        sizeref=2.*max(size)/(40.**2.1),
        sizemin=4,
        color=hourly_df['avg']
    ), 
    name='Average G-FORCE each hr on [date]')],
    layout=layout
)

fig.update_xaxes(
    rangeslider_visible=True
)

fig.show()

### Bubbles on specific date - time

In [660]:
layout = go.Layout(
    title='g-force over time on specific date',
    plot_bgcolor='rgba(230, 230, 230, 0.2)',
    showlegend=True
)
size=pd.DataFrame(days['16-05-2020'])['G-Force']

fig = go.Figure(data=[go.Scatter(
    x=pd.DataFrame(days['16-05-2020'])['Time'],
    y=pd.DataFrame(days['16-05-2020'])["G-Force"],
    mode='markers',
    marker=dict(
        size=size,
        sizemode='area',
        sizeref=2.*max(size)/(40.**2),
        sizemin=4,
        color=pd.DataFrame(days['16-05-2020'])['G-Force']
    ), 
    name='G-FORCE')],
    layout=layout
)

fig.update_xaxes(
    rangeslider_visible=True
)

fig.show()

### Bubbles specific date - time (same as ^? )

In [657]:
fig = px.scatter(pd.DataFrame(days['16-05-2020']), x="Time", y="G-Force", color="G-Force", title="gf vs. t",
                 size='G-Force', hover_data=['G-Force'])
fig.show()

### Bubbles avg, count, hrs

In [169]:
size = hourly_df['count']

fig = px.scatter(hourly_df, x="time", y="avg", color="avg", title="avg / size=count v. hr",
                 size=size, hover_data=['count'])


fig.update_traces(marker=dict(size=size,
                              sizemode='area',
                            sizeref=3.*max(size)/(40.**2.4),
                            sizemin=4))
                 
fig.show()

### Bubbles avg, count, hrs - other size tho

In [678]:
size = hourly_df['count']

fig = px.scatter(hourly_df, x="time", y="count", color="count", title="count vs. hr / size=avg ",
                 size=size, hover_data=['avg'])


fig.update_traces(marker=dict(size=size,
                              sizemode='area',
                            sizeref=2.*max(size)/(40.**2.3),
                            sizemin=4))
                 
fig.show()

In [693]:
fig.write_html("C:/Users/Math_/OneDrive/Dokumenter/Arb/Test.html", include_plotlyjs='cdn', 
               include_mathjax='cdn', full_html=False)

In [688]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)  

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### TB fixed

In [308]:
animation_list = hourly_df['count'].tolist() 
fig = px.scatter(hourly_df, x='avg', y='count', animation_frame='time', animation_group=animation_list,
           size='count', color='count', hover_name='count',
           log_x=True, size_max=10, range_y=[1,3])
fig

### Animation G-Force Time on specific date

In [249]:
g_f_list = pd.DataFrame(days['16-05-2020'])['G-Force'].tolist()
time_list = pd.DataFrame(days['16-05-2020'])['Time'].tolist()
size=pd.DataFrame(days['16-05-2020'])['G-Force']

trace1 = go.Scatter(x=pd.DataFrame(days['16-05-2020'])['Time'],
                    y=g_f_list,
                    name='gf',
                    mode='markers',
                    marker=dict(
                        size=size,
                        sizemode='area',
                        sizeref=2.*max(size)/(40.**2.1),
                        sizemin=4,
                        color=g_f_list,
                        line=dict(width=1, color="#CC0066")
                    ), 
                    line=dict(width=1.5))

frames = [dict(data= [dict(type='scatter',
                           x=df['Time'][:k+1],
                           y=g_f_list[:k+1])],
               traces = [0, 1],  #this means that  frames[k]['data'][0]  updates trace1, and   frames[k]['data'][1], trace2 
              )for k  in  range(1, len(g_f_list)-1)] 

layout = go.Layout(showlegend=True,
                   hovermode='closest', 
                   title='G-Force vs. Time on specific date',
                   updatemenus=[dict(type='buttons', showactive=False,
                                y=1.05,
                                x=1.15,
                                xanchor='left',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=4, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])])])

fig = go.Figure(data=[trace1], frames=frames, layout=layout)
fig.show()

### Bubbles, avg, count, hrs, animation (so and so)

In [173]:
g_f_list = hourly_df['avg'].tolist()
time_list = hourly_df['time'].tolist()
size=hourly_df['count'] #whichever

trace1 = go.Scatter(x=hourly_df['time'],
                    y=g_f_list,
                    name='gf',
                    mode='markers',
                    marker=dict(
                        size=size,
                        sizemode='area',
                        sizeref=2.*max(size)/(40.**2.3),
                        sizemin=4,
                        color=g_f_list,
                        line=dict(width=1, color="#CC0066")
                    ), 
                    line=dict(width=1.5))

frames = [dict(data= [dict(type='scatter',
                           x=hourly_d['time'][:k+1],
                           y=g_f_list[:k+1])],
               traces = [0, 1],  #this means that  frames[k]['data'][0]  updates trace1, and   frames[k]['data'][1], trace2 
              )for k  in  range(1, len(g_f_list)-1)] 

layout = go.Layout(showlegend=True,
                   hovermode='closest', 
                   updatemenus=[dict(type='buttons', showactive=False,
                                y=1.05,
                                x=1.15,
                                xanchor='left',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=4, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])])])

fig = go.Figure(data=[trace1], frames=frames, layout=layout)
fig.show()

### TB fixed as well

### Animation over all counts and averages 

In [221]:
g_f_list = hourly_df['count'].tolist()
time_list = hourly_df['time'].tolist()
size=hourly_df['avg']
test = 5

trace1 = go.Scatter(x=hourly_df['time'],
                    y=g_f_list,
                    name='Average G-Force',
                    mode='markers',
                    #text=hourly_df['avg'],
                    hovertemplate='<i>Average<i>: somehow get variable in here...',
                    marker=dict(
                        size=size,
                        sizemode='area',
                        sizeref=2.*max(size)/(40.**2),
                        sizemin=4,
                        color=size,
                        line=dict(width=1, color="#CC0066")
                    ), 
                    line=dict(width=1.5))

frames = [dict(data= [dict(type='scatter',
                           x=hourly_df['time'][:k+1],
                           y=g_f_list[:k+1])],
               traces = [0, 1],  #this means that  frames[k]['data'][0]  updates trace1, and   frames[k]['data'][1], trace2 
              )for k  in  range(1, len(g_f_list)-1)] 

layout = go.Layout(showlegend=True,
                   hovermode='closest', 
                    xaxis_title="Hour (00.00-23.00)",
                    yaxis_title="Count pr. hour",
                   updatemenus=[dict(type='buttons', showactive=False,
                                y=1.05,
                                x=1.15,
                                xanchor='left',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=4, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])])])

fig = go.Figure(data=[trace1], frames=frames, layout=layout)
fig.show()

### Basically same as ^just no animation. (full but w/ hrs)

In [247]:
size = hourly_df['avg']

fig = px.scatter(hourly_df, x="time", y="count", color="avg", title="count / size=avg vs. hr",
                 size=size, hover_data=['avg'])


fig.update_traces(marker=dict(size=size,
                              sizemode='area',
                            sizeref=1.5*max(size)/(40.**2.1)))

fig.update_layout(hovermode='y',
        hoverlabel=dict(
        bgcolor="white", 
        font_size=14, 
        font_family="Cambria"
        ), 
        xaxis_title="Hour (00.00-23.00)",
        yaxis_title="Count pr. hour"
     )
fig.show()

In [676]:
fig = px.bar(df, x="Time", y="G-Force", color="G-Force",
  animation_frame="Date", animation_group="G-Force", range_y = [1.1,3])
fig.show()

### Okay... just for funsies

In [675]:
x = hourly_df['time']
y = hourly_df['avg']
z = hourly_df['count']

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
    size=15,
    color=hourly_df['count'],
    colorscale='Viridis',
    opacity=0.8),
)

data = [trace1]

layout = go.Layout(
    margin=dict(
    l=0,
    r=0,
    b=0,
    t=0)
)

fig = go.Figure(data=data, layout=layout)

fig.update_layout(scene = dict(
    xaxis = dict(title='Time', backgroundcolor="rgb(200,200,230)"),
    yaxis = dict(title='Average', backgroundcolor="rgb(230,200,230)"),
    zaxis = dict(title='Count', backgroundcolor="rgb(230,230,200)")),
    width=700,
    title="Cool 3D Stuff Woah",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f"
    )
)

pyo.iplot(fig, filename='3d-data-plot')

In [723]:
from kafka import KafkaConsumer
from json import loads
from IPython.display import Javascript
from IPython.display import clear_output

sasl_mechanism = "PLAIN"
security_protocol = "SASL_PLAINTEXT"

# Local
ip = '192.168.0.185'
port = "9092"
username = "northtech"
password = "northtech-secret"

# change mode between (test / prod / local) kafka
mode = "test"

#93.160.21.204
if mode == "prod":
    ip = '192.168.0.25'
    port = "9092"
    username = "northtech"
    password = "northtech-secret"

if mode == "test":
    ip = '192.168.0.25'
    port = "9192"
    username = "admin"
    password = "admin"
    
if mode == 'yikes':
    ip = '192.168.64.10'
    port = "31526"
    
connection = '{0}:{1}'.format(ip, port)

consumer = KafkaConsumer(
    'KU',
    bootstrap_servers=[connection],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    security_protocol=security_protocol,
    #ssl_context=context,
    #ssl_check_hostname=True,
    sasl_mechanism = sasl_mechanism,
    sasl_plain_username = username,
    sasl_plain_password = password
    )    

mac_address = ''
temp = []

"""
for message in consumer:
    if mac_address is '':
        #mac_address = list(message.value.keys())[0]
        mac_address = message.key
    print(dict(message.value))
    temp.append(message.value)
    #clear_output(wait=True)
    #print(temp)
    #df = pd.DataFrame(temp)
    #display(df)
"""

for message in consumer:
    #print(message.value)
    if mac_address is '':
        #mac_address = list(message.value.keys())[0]
        mac_address = message.key
    #temp.append(message.value.get(mac_address))
    for e in message.value:
        temp.append(e)
    clear_output(wait=True)
    df = pd.DataFrame(temp)
    display(df)

TimeStamp    Gforce  BatteryVoltage  Temperature
0     1592392306  1.004692        4.907491     27.79669
1     1592392307  1.006032        4.882402     27.78596
2     1592392307  1.004399        4.907491     27.79669
3     1592392308  1.005999        4.897456     27.81813
4     1592392308  1.004793        4.897456     27.82886
...          ...       ...             ...          ...
1838  1592558607  1.018117        4.897456     31.47539
1839  1592558608  1.014609        4.932581     31.46467
1840  1592558608  1.014365        4.892437     31.47539
1841  1592558609  1.009879        4.912509     31.46467
1842  1592558609  1.016867        4.907491     31.46467

[1843 rows x 4 columns]

KeyboardInterrupt: 

In [724]:
size = df['Gforce']

fig = px.scatter(df, x="TimeStamp", y="Gforce", color="Gforce", title="--",
                 size=size, hover_data=['Gforce'])


fig.update_traces(marker=dict(size=size,
                              sizemode='area',
                            sizeref=1.5*max(size)/(40.**2.1)))

fig.update_layout(hovermode='y',
        hoverlabel=dict(
        bgcolor="white", 
        font_size=14, 
        font_family="Cambria"
        ), 
        xaxis_title="Timestamp",
        yaxis_title="G-Force"
     )

py.iplot(fig)